## Clip Dynamic GT and nuScenes

In [75]:
import cv2
import numpy as np
import os

In [ ]:
import glob

type = 'static'
# type = 'dynamic'
model_input_folder = '/data/nuscenes/nuscenes_mini/samples/CAM_FRONT'
model_output_folder = '/home/pak/vkr'
model_gt_folder = f'/home/pak/vkr/data/datasets/nuscenes/nuscenes_mini/{type}_gt'

In [ ]:
sorted_input_names = sorted(glob.glob(f'{model_input_folder}/**.jpg', recursive=True))

image_sample = cv2.imread(sorted_input_names[0])

height = image_sample.shape[0]
width = image_sample.shape[0]

size = (image_sample.shape[1] + image_sample.shape[0], image_sample.shape[0])

In [ ]:
fps = 5
out = cv2.VideoWriter(model_output_folder + '/fetched.avi', cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

for img_index, img_path in enumerate(sorted_input_names):    
    input_image = cv2.imread(img_path)
    gt_image = cv2.imread(img_path.replace('samples/CAM_FRONT', f'{type}_gt'))
        
    gt_image = cv2.resize(
        gt_image, dsize=(height, width), interpolation=cv2.INTER_CUBIC
    )
    data = np.concatenate((input_image, gt_image), 1)
    out.write(data)
out.release()

## Clip static+dynamic video

In [ ]:
sorted_input_names = sorted(glob.glob(f'{model_input_folder}/**.jpg', recursive=True))
sorted_static_names = sorted(glob.glob(f'{model_output_folder}/static/**.png', recursive=True))

image_sample = cv2.imread(sorted_input_names[0])
static_sample = cv2.imread(sorted_static_names[0])
dynamic_sample = cv2.imread(sorted_static_names[0].replace('static', 'dynamic'))

height = image_sample.shape[0]
width = image_sample.shape[0]

size = (image_sample.shape[1] + image_sample.shape[0], image_sample.shape[0])

In [ ]:
fps = 5
out = cv2.VideoWriter(model_output_folder + '/fetched.avi', cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

for img_index, img_path in enumerate(sorted_static_names):
    assert img_path.split('_')[-1][:-3] == sorted_input_names[img_index].split('_')[-1][:-3]
    
    input_image = cv2.imread(sorted_input_names[img_index])
    static_image = cv2.imread(img_path)
    dynamic_image = cv2.imread(img_path.replace('static', 'dynamic'))
    
    dynamic_image[dynamic_image == 255] = 128
    
    static_image = cv2.resize(
        static_image, dsize=(height, width), interpolation=cv2.INTER_CUBIC
    )
    dynamic_image = cv2.resize(
        dynamic_image, dsize=(height, width), interpolation=cv2.INTER_CUBIC
    )
    data = np.concatenate((input_image, dynamic_image + static_image), 1)
    out.write(data)
out.release()

## Make train-val split (v1.0mini all_samples in train and val)

In [ ]:
import os
import glob

In [ ]:
model_gt_folder = '/home/pak/vkr/data/datasets/nuscenes/nuscenes_mini/dynamic_gt'

arr = sorted(glob.glob(f'{model_gt_folder}/**.jpg', recursive=True))

In [ ]:
arr1 = ['static' + x[58:] for x in arr]

In [ ]:
with open('/home/pak/vkr/monolayout/splits/nuscenes/train_files.txt', 'w') as outfile:
    for x in arr1:
        outfile.write(x)
        outfile.write('\n')

with open('/home/pak/vkr/monolayout/splits/nuscenes/valid_files.txt', 'w') as outfile:
    for x in arr1:
        outfile.write(x)
        outfile.write('\n')